# Init Setup

## Checking the environment

In [ ]:
import sys
print(sys.executable)

## For nvidia GPU (AMD and Intel do not ned this step)

In [ ]:
!nvidia-smi

## Installation

In [ ]:
!pip install tensorflow-cpu==2.10
!pip install tensorflow-directml-plugin
!pip install "numpy<2.0"
!pip install pandas
!pip install joblib
!pip install scikit-learn
!pip install fuzzywuzzy
!pip install nltk
!pip install matplotlib
!pip install seaborn
!pip install tabulate
!pip install rouge_score
!pip install pyyaml
!pip install pulp
!pip install ipywidgets
!pip install python-Levenshtein
!pip install autocorrect

## Install nltk packages

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

## Check tensorflow version

In [ ]:
import tensorflow as tf
print("✅ TensorFlow version:", tf.__version__)

## Check tensorflow devices checking

In [ ]:
import tensorflow as tf
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU Available:", tf.config.list_physical_devices('GPU'))


## Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import joblib
import tensorflow as tf

# Clean Data (V2)

## Load and Inspect Dataset

In [ ]:
import pandas as pd

# Load data
df = pd.read_csv("food_data_dataset.csv")

# Inspect structure
print(df.head())
print(df.info())
print(df.describe())
print("Missing values:\n", df.isnull().sum())


## Basic Cleaning & Missing Values

In [ ]:
# Fill missing values
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(df[col].mean())  # numeric: mean
    else:
        df[col] = df[col].fillna(df[col].mode()[0])  # categorical: mode


## Outlier Clipping (IQR Method)

In [ ]:
import numpy as np

numerical_cols = df.select_dtypes(include=np.number).columns
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)


## Normalize Nutrients (Optional, but recommended)

In [ ]:
if 'Serving_Size_g' in df.columns:
    df[numerical_cols] = df[numerical_cols].div(df['Serving_Size_g'], axis=0) * 100


## Clean the Food_Item Text

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces
    return text

df['Food_Item'] = df['Food_Item'].astype(str).apply(clean_text)


## Label Creation

In [ ]:
# Convert Nutrition_Density to categories
df['Meal_Type'] = pd.qcut(df['Nutrition_Density'], q=3, labels=['Low', 'Medium', 'High'])


## TF-IDF Vector for Text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

tfidf = TfidfVectorizer()
X_text = tfidf.fit_transform(df['Food_Item']).toarray()
joblib.dump(tfidf, "tfidf_vectorizer.pkl")


## Standardize Numerical Features

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_cols = [col for col in df.columns if col not in ['Food_Item', 'Meal_Type']]
df[numeric_cols] = df[numeric_cols].fillna(0)

scaler = StandardScaler()
X_num = scaler.fit_transform(df[numeric_cols])
joblib.dump(scaler, "nutrition_scaler.pkl")


## Combine Text + Numeric Features

In [ ]:
import numpy as np

X = np.hstack([X_text, X_num])


## Encode Labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Meal_Type'])
joblib.dump(label_encoder, "label_encoder.pkl")


## Stratified Train-Test Split

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]


## Save Cleaned Dataset (Optional)

In [ ]:
df.to_csv("cleaned_food_data.csv", index=False)


# Train Data With Tensorflow

## Train Data With Tensorflow (V17)

In [ ]:
import os
import json
import logging
import yaml
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_recall_fscore_support,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, log_loss
)
from sklearn.calibration import calibration_curve
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint,
    ReduceLROnPlateau, TensorBoard
)
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# --- Configuration ---
CONFIG_PATH = "config.yaml"
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

def load_config(path):
    if os.path.exists(path):
        return yaml.safe_load(open(path))
    return {
        'data_path': 'cleaned_food_data.csv',
        'text_column': 'Food_Item',
        'target_column': 'Meal_Type',
        'numeric_columns': [],
        'tfidf__ngram_range': (1,2),
        'tfidf__max_features': 1500,
        'validation_split': 0.2,
        'model': {
            'layers': [1024, 512, 256],
            'dropouts': [0.4, 0.3, 0.2],
            'l2': 0.001,
            'learning_rate': 1e-3,
            'batch_size': 32,
            'epochs': 100
        },
        'cv_splits': 5,
        'grid_search': False,
        'gen_refs_path': 'generation_refs.json',
        'gen_hyps_path': 'generation_hyps.json',
        'feedback_path': 'user_feedback.csv'
    }

config = load_config(CONFIG_PATH)

# === Data Loading & Cleaning ===
logging.info("Loading and cleaning data...")

def load_and_clean(path, text_col):
    df = pd.read_csv(path)
    df[text_col] = (
        df[text_col].astype(str)
        .str.lower()
        .str.replace(r"\(.*?\)", "", regex=True)
        .str.replace(r"[^a-zA-Z\s]", "", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )
    return df

# Load and clean
df = load_and_clean(config['data_path'], config['text_column'])

# === Label Creation ===
logging.info("Creating quantile-based labels...")
labels = ['Low', 'Medium', 'High']
df[config['target_column']] = pd.qcut(
    df['Nutrition_Density'], q=3, labels=labels
)

# Fill numeric missing
num_cols = [c for c in df.columns if c not in [config['text_column'], config['target_column']]]
df[num_cols] = df[num_cols].fillna(0)
config['numeric_columns'] = num_cols

# Encode targets
label_enc = LabelEncoder()
y = label_enc.fit_transform(df[config['target_column']])
joblib.dump(label_enc, 'label_encoder.pkl')

# Preprocessing pipelines
text_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=config['tfidf__ngram_range'],
        max_features=config['tfidf__max_features']
    ))
])
num_pipeline = Pipeline([('scaler', StandardScaler())])
preprocessor = ColumnTransformer([
    ('text', text_pipeline, config['text_column']),
    ('num', num_pipeline, num_cols)
], sparse_threshold=0)

# Build Keras model
def build_model(input_dim, cfg):
    inp = Input(shape=(input_dim,))
    x = inp
    for units, drop in zip(cfg['layers'], cfg['dropouts']):
        x = Dense(units,
                  activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(cfg['l2']))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop)(x)
    out = Dense(len(label_enc.classes_), activation='softmax')(x)
    model = Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(cfg['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Assemble full pipeline
clf = Pipeline([
    ('preproc', preprocessor),
    ('keras', KerasClassifier(
        build_fn=lambda: build_model(
            input_dim=preprocessor.fit_transform(df).shape[1],
            cfg=config['model']
        ),
        epochs=config['model']['epochs'],
        batch_size=config['model']['batch_size'],
        validation_split=config['validation_split'],
        verbose=1,
        callbacks=[
            EarlyStopping('val_loss', patience=10, restore_best_weights=True),
            ReduceLROnPlateau('val_loss', factor=0.5, patience=4, min_lr=1e-6),
            ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True),
            TensorBoard(log_dir=f"logs/v17_{datetime.now():%Y%m%d_%H%M%S}")
        ]
    ))
])

# Optional grid search
if config['grid_search']:
    param_grid = {
        'keras__epochs': [50, 100],
        'keras__batch_size': [32, 64]
    }
    cv = StratifiedKFold(n_splits=config['cv_splits'], shuffle=True, random_state=SEED)
    clf = GridSearchCV(clf, param_grid, cv=cv, scoring='accuracy')

# Train/test split
df_features = df.copy()
X_train, X_test, y_train, y_test = train_test_split(
    df_features, y, test_size=0.2, stratify=y, random_state=SEED
)

# Training
logging.info("Training Version 17 model...")
clf.fit(X_train, y_train)
best = clf.best_estimator_ if hasattr(clf, 'best_estimator_') else clf

# Predictions & Evaluation
X_t = best.named_steps['preproc'].transform(X_test)
y_probs = best.named_steps['keras'].model.predict(X_t)
y_pred = np.argmax(y_probs, axis=1)

# Classification report
print("===== Classification Report =====")
print(classification_report(y_test, y_pred, target_names=label_enc.classes_))

# Per-class metrics
prf = precision_recall_fscore_support(y_test, y_pred, zero_division=0)
df_metrics = pd.DataFrame({
    'precision': prf[0],
    'recall': prf[1],
    'f1_score': prf[2],
    'support': prf[3]
}, index=label_enc.classes_)
print("\n===== Per-Class Metrics =====")
print(df_metrics)

# Overall metrics
overall_acc = accuracy_score(y_test, y_pred) * 100
overall_prec = precision_score(y_test, y_pred, average='macro', zero_division=0) * 100
overall_rec = recall_score(y_test, y_pred, average='macro', zero_division=0) * 100
overall_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0) * 100

print(f"\nOverall Accuracy: {overall_acc:.2f}%")
print(f"Overall Precision (macro): {overall_prec:.2f}%")
print(f"Overall Recall (macro): {overall_rec:.2f}%")
print(f"Overall F1-score (macro): {overall_f1:.2f}%")
print(f"Log Loss: {log_loss(y_test, y_probs):.4f}")
print(f"ROC-AUC Score: {roc_auc_score(tf.keras.utils.to_categorical(y_test), y_probs, multi_class='ovr'):.4f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d',
            xticklabels=label_enc.classes_,
            yticklabels=label_enc.classes_)
plt.title('Confusion Matrix v17')
plt.savefig('confusion_matrix.png')
plt.close()

# === Visualization Updates v17 ===
# 1) Violin plot of predicted probabilities by true class
prob_df = pd.DataFrame(y_probs, columns=label_enc.classes_)
prob_df['true_class'] = label_enc.inverse_transform(y_test)
prob_melt = pd.melt(
    prob_df,
    id_vars='true_class',
    var_name='predicted_class',
    value_name='probability'
)
plt.figure(figsize=(10,6))
sns.violinplot(
    x='predicted_class',
    y='probability',
    hue='true_class',
    data=prob_melt,
    split=True
)
plt.title('Predicted Probability Distributions by Class (v17)')
plt.ylabel('Probability')
plt.xlabel('Predicted Class')
plt.legend(title='True Class')
plt.savefig('probability_violin.png')
plt.close()

# 2) Calibration curve per class
plt.figure(figsize=(8, 6))
for idx, cls in enumerate(label_enc.classes_):
    true_bin = (y_test == idx).astype(int)
    prob_cls = y_probs[:, idx]
    prob_true, prob_pred = calibration_curve(true_bin, prob_cls, n_bins=10)
    plt.plot(prob_pred, prob_true, marker='o', label=cls)
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.title('Calibration Curve by Class (v17)')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.legend()
plt.savefig('calibration_curve.png')
plt.close()

# Generation metrics (BLEU, ROUGE)
if os.path.exists(config['gen_refs_path']) and os.path.exists(config['gen_hyps_path']):
    with open(config['gen_refs_path']) as rf, open(config['gen_hyps_path']) as hf:
        refs = json.load(rf)
        hyps = json.load(hf)
    refs_tok = [[r.split() for r in rlist] for rlist in refs]
    hyps_tok = [h.split() for h in hyps]
    bleu = corpus_bleu(refs_tok, hyps_tok, smoothing_function=SmoothingFunction().method4)
    scorer = rouge_scorer.RougeScorer(['rouge1','rougeL'], use_stemmer=True)
    r1=[]; rL=[]
    for rlist, hyp in zip(refs, hyps):
        sc = scorer.score(" ".join(rlist), hyp)
        r1.append(sc['rouge1'].fmeasure)
        rL.append(sc['rougeL'].fmeasure)
    print(f"BLEU: {bleu:.4f}, ROUGE-1 F1: {np.mean(r1):.4f}, ROUGE-L F1: {np.mean(rL):.4f}")
else:
    logging.warning("Skipping generation metrics.")

# Feedback analysis
if os.path.exists(config['feedback_path']):
    fb = pd.read_csv(config['feedback_path'])
    for col in ['usability_score', 'satisfaction_score']:
        if col in fb:
            print(f"{col}: mean={fb[col].mean():.2f}, median={fb[col].median():.2f}")
else:
    logging.warning("Skipping feedback analysis.")

# Save artifacts
logging.info("Saving V17 artifacts...")
best.named_steps['keras'].model.save('food_nutrition_model.keras')
joblib.dump(label_enc.classes_.tolist(), 'class_names.pkl')
joblib.dump(preprocessor, 'preprocessor.pkl')
with open('version_info.json', 'w') as vf:
    json.dump({'version': '17', 'timestamp': str(datetime.now())}, vf)
logging.info("✅ Version 17 pipeline complete.")

# References:
# - Pandas: https://pandas.pydata.org/
# - NumPy: https://numpy.org/
# - Scikit-learn: https://scikit-learn.org/stable/
# - TensorFlow Keras: https://www.tensorflow.org/guide/keras/
# - Seaborn Violin: https://seaborn.pydata.org/generated/seaborn.violinplot.html
# - Calibration Curve: https://scikit-learn.org/stable/modules/calibration.html
# - Matplotlib: https://matplotlib.org/
# - NLTK BLEU: https://www.nltk.org/api/nltk.translate.html
# - rouge_score: https://pypi.org/project/rouge-score/
# - joblib: https://joblib.readthedocs.io/
# - PyYAML: https://pyyaml.org/


# ChatBot Part

## Chatbot (V21)

In [ ]:
import os
import random
import re
import logging
import difflib
import time
import numpy as np
import pandas as pd
import nltk
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
import joblib
from rich.console import Console
from rich.spinner import Spinner
from rich.text import Text
from autocorrect import Speller

# === NLTK Resources ===
nltk.download('wordnet')
nltk.download('omw-1.4')

# === Logging Setup ===
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s [%(levelname)s] %(message)s')
console = Console()

# === Autocorrect Setup ===
spell = Speller(lang='en')

# === Paths & Artifacts ===
MODEL_PATH        = 'food_nutrition_model.keras'
PREPROCESSOR_PATH = 'preprocessor.pkl'
LABELENC_PATH     = 'label_encoder.pkl'
DATA_PATH         = 'cleaned_food_data.csv'

# === Load model and data ===
with console.status("Loading model and data...", spinner="dots"):
    model     = load_model(MODEL_PATH)
    preproc   = joblib.load(PREPROCESSOR_PATH)
    label_enc = joblib.load(LABELENC_PATH)
    data      = pd.read_csv(DATA_PATH)

# === Small-talk Patterns & Responses ===
GREETINGS_PATTERN = re.compile(r"\b(hi|hello|hey|how are you|good morning|good afternoon|good evening)\b", re.I)
THANKS_PATTERN    = re.compile(r"\b(thanks|thank you|thx)\b", re.I)
FAREWELL_PATTERN  = re.compile(r"\b(bye|goodbye|see you|farewell)\b", re.I)

GREETING_RESPONSES = [
    "Hello there! 🥦 How can I help you with your nutrition today?",
    "Hi! I'm here to chat about food and nutrition. What would you like to know?",
    "Hey! Want to know the meal type or nutrition facts of something?"
]
THANK_RESPONSES = [
    "You're welcome! 🍽️",
    "Anytime! Let me know if you have more questions.",
    "Glad to help!"
]
FAREWELL_RESPONSES = [
    "Goodbye! Stay healthy! 🌱",
    "See you later! Keep eating well!",
    "Farewell! Remember to balance your meals."
]

# === Prepare food lookup ===
data['food_lower'] = data['Food_Item'].str.lower()
food_list = data['food_lower'].tolist()
food_map  = {row['food_lower']: row.drop('food_lower').to_dict() for _, row in data.iterrows()}
vectorizer = TfidfVectorizer().fit(food_list)
vectors    = vectorizer.transform(food_list)
lem = WordNetLemmatizer()

# === Utility Functions ===
def get_synonyms(word):
    return {lm.name().lower() for syn in wordnet.synsets(word) for lm in syn.lemmas()}

def semantic_match(text, thresh=0.2):
    v = vectorizer.transform([text])
    sims = cosine_similarity(v, vectors).flatten()
    idx  = sims.argmax()
    return food_list[idx] if sims[idx] >= thresh else None

def fuzzy_suggestions(text, n=3, cutoff=60):
    scores = [(name, fuzz.token_set_ratio(text, name)) for name in food_list]
    filtered = sorted([p for p in scores if p[1] >= cutoff], key=lambda x: -x[1])
    return [name for name, score in filtered[:n]]

def normalize(tok):
    return lem.lemmatize(tok.lower(), pos='n')

def autocorrect_text(text):
    words = re.findall(r'\w+', text)
    corrected_words = [spell(word) for word in words]
    corrected_text = ' '.join(corrected_words)
    if corrected_text != text:
        console.print(f"[yellow]Autocorrected input: '{text}' → '{corrected_text}'[/yellow]")
    return corrected_text

# === Similarity-based Suggestions ===
def suggest_similar_foods(food_key, top_n=5):
    if food_key not in food_list:
        return []
    idx = food_list.index(food_key)
    sims = cosine_similarity(vectors[idx], vectors).flatten()
    top_idx = np.argsort(sims)[-top_n-1:-1][::-1]
    return [food_list[i] for i in top_idx]

# === Extraction Logic ===
def extract_single_food_and_qty(text):
    txt = text.lower()
    txt = autocorrect_text(txt)  # <-- Autocorrect here
    qty = 100.0
    m = re.search(r"(\d+(?:\.\d+)?)\s*(g|gram|kg|oz)", txt)
    if m:
        val, unit = float(m.group(1)), m.group(2)
        qty = val * 1000 if 'kg' in unit else (val*28.35 if 'oz' in unit else val)
    for name in food_list:
        if name in txt:
            return name, qty
    suggestions = fuzzy_suggestions(txt)
    if suggestions:
        if len(suggestions) == 1:
            return suggestions[0], qty
        console.print(f"[yellow]I found multiple close matches for '{text}':[/yellow]")
        for idx, opt in enumerate(suggestions, 1):
            console.print(f"  {idx}. {opt}")
        choice = console.input("Enter the number of the correct item, or 0 to skip: ")
        console.print(f"[blue]You:[/blue] {choice}")
        if choice.isdigit() and 1 <= int(choice) <= len(suggestions):
            return suggestions[int(choice)-1], qty
    sem = semantic_match(txt)
    if sem:
        console.print(f"[yellow]Semantically matched '{text}' to '{sem}'[/yellow]")
        return sem, qty
    for tok in re.findall(r"\w+", txt):
        for syn in get_synonyms(tok):
            if syn in food_list:
                console.print(f"[yellow]Using synonym '{syn}' for '{tok}'[/yellow]")
                return syn, qty
    return None, qty

def extract_foods_and_qty(user_text):
    clauses = re.split(r',| and ', user_text)
    results = []
    for clause in clauses:
        name, qty = extract_single_food_and_qty(clause)
        if name:
            results.append((name, qty))
    return results

# === Prediction & Animated Response ===
def predict_and_respond(food_key, qty):
    record = food_map.get(food_key)
    if not record:
        console.print(f"[red]Sorry, I couldn't find that food: {food_key}[/red]")
        return
    nutrition = {k: v for k, v in record.items() if isinstance(v, (int, float))}
    with console.status("Analyzing nutrition...", spinner="bouncingBar"):
        time.sleep(1.5)
        df_input = pd.DataFrame([{**nutrition, 'Food_Item': record['Food_Item']}])
        X = preproc.transform(df_input)
        probs = model.predict(X, verbose=0)[0]
        idx = np.argmax(probs)
        meal = label_enc.inverse_transform([idx])[0]
        conf = probs[idx]
    console.print(f":plate_with_cutlery: [bold green]{qty:.0f}g {record['Food_Item']}[/bold green] "
                  f"→ [cyan]{meal}[/cyan] ([magenta]{conf*100:.1f}%[/magenta])")
    console.print("[underline]Nutrition per serving:[/underline]")
    for k, v in nutrition.items():
        console.print(f"- {k}: [bold]{v * qty/100:.2f}[/bold]")

# === Main CLI Loop v21 ===
def chatbot():
    console.print(":seedling: [bold]Welcome to Smart Food NutriBot v21[/bold] (type 'exit' to quit)\n")
    while True:
        user = console.input("[blue]You:[/blue] ").strip()
        console.print(f"[blue]You:[/blue] {user}")
        if user.lower() == 'exit':
            console.print(f"[bold yellow]{random.choice(FAREWELL_RESPONSES)}[/bold yellow]")
            break
        if GREETINGS_PATTERN.search(user):
            console.print(f"[green]{random.choice(GREETING_RESPONSES)}[/green]")
            continue
        if THANKS_PATTERN.search(user):
            console.print(f"[green]{random.choice(THANK_RESPONSES)}[/green]")
            continue
        if FAREWELL_PATTERN.search(user):
            console.print(f"[green]{random.choice(FAREWELL_RESPONSES)}[/green]")
            continue
        if re.search(r"\b(suggest|recommend)\b", user.lower()):
            name, _ = extract_single_food_and_qty(user)
            if name:
                record = food_map.get(name)
                sim_list = suggest_similar_foods(name)
                if sim_list:
                    console.print(f":mag: Here are some foods similar to [bold]{record['Food_Item']}[/bold]:")
                    for item in sim_list:
                        console.print(f"- {food_map[item]['Food_Item']}")
                else:
                    console.print(f"[red]Sorry, I don't have suggestions for {record['Food_Item']}.[/red]")
                continue
        foods = extract_foods_and_qty(user)
        if not foods:
            console.print("[red]I couldn't identify any food items in your query. Could you rephrase?[/red]")
            continue
        for food_key, qty in foods:
            predict_and_respond(food_key, qty)

if __name__ == '__main__':
    chatbot()
